In [4]:
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
from numba import jit

In [8]:
def get_Matrix(rating_path):
        csv_df = pd.read_csv(rating_path)
        users_size = np.max(csv_df['userId'])
        movies_size = np.max(csv_df['movieId'])
        
        rating_M = np.zeros((users_size, movies_size))
        
        for row in csv_df.itertuples():
            rating_M[row[1]-1,row[2]-1] = row[3]

        return train_test_split(rating_M)
    
    
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]

    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test


train, test = get_Matrix('dataset/rating.csv')
print(train.shape, test.shape)

(610, 193609) (610, 193609)


In [11]:
@jit
class MemoryBCF:
    def __init__(self, train, test, type='user'):
        self.type = type
        self.rating_train_M = train
        self.rating_test_M = test
        self.result = self.evaluate()
    @jit
    def _sim(self, ratings, epsilon=1e-9):
        if self.type == 'user':
            sim = ratings.dot(ratings.T) + epsilon
        elif self.type == 'item':
            sim = ratings.T.dot(ratings) + epsilon
        norms = np.array([np.sqrt(np.diagonal(sim))])
        return (sim / norms / norms.T)

    @jit
    def _predict(self, ratings, similarity):
    
#     nzero = ratings[user].nonzero()[0]
#     baseline = item_mean + user_mean[user] - all_mean
#     choice = nzero[item_sim[item, nzero].argsort()[::-1][:k]]
#     prediction = ratings[user, choice].dot(item_sim[item, choice])                / sum(item_sim[item, choice]) + baseline[item]
#     if prediction > 5: prediction = 5
#     if prediction < 1: prediction = 1
#     return prediction 



#         if self.type == 'user':
#             pred = np.zeros(ratings.shape)
#             for i in range(ratings.shape[0]):
#                 for j in range(ratings.shape[1]):
#                     pred[i, j] = similarity[i, :].dot(ratings[:, j])\
#                                  /np.sum(np.abs(similarity[i, :]))
#                 print(i)
            
#             return pred

        
        pred = np.zeros(ratings.shape)
        k = 40
        if self.type == 'user':
            for i in range(ratings.shape[0]):
                top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
                for j in range(ratings.shape[1]):
                    pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                    pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
                print(i)    
            return pred
        
        elif self.type == 'item':
            pred = ratings.dot(similarity) / \
                np.array([np.abs(similarity).sum(axis=1)])
            return np.array(pred)
    @jit
    def _rmse(self, pred, actual):
        #actual = actual.values
        pred = pred[actual.nonzero()].flatten()
        actual = actual[actual.nonzero()].flatten()
        return np.sqrt(mean_squared_error(pred, actual))
    @jit
    def evaluate(self):
        sim = self._sim(self.rating_train_M)
        print(sim.shape)
        pred = self._predict(self.rating_train_M, sim)
        print(pred.shape)
        rmse_val = self._rmse(pred, self.rating_test_M)
        print(rmse_val)
        return rmse_val

AttributeError: type object 'MemoryBCF' has no attribute '__defaults__'

In [ ]:
UserBM = MemoryBCF(train, test, 'user')
print(f'User based RMSE: {UserBM.result}')

(610, 610)


/Users/xuanxinfang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


0
